In [ ]:
!pip install "paho-mqtt<2.0.0"

In [ ]:
import threading
import random
import time
import paho.mqtt.client as mqtt

class MQTTClient:
    def __init__(self, client_name, broker_address, port, username, password, topic):
        """
        Initializes an MQTTClient instance.

        Args:
            client_name (str): Name of the MQTT client.
            broker_address (str): Address of the MQTT broker.
            port (int): Port number of the MQTT broker.
            username (str): Username for authentication.
            password (str): Password for authentication.
            topic (str): Topic to which heartbeats will be published.
        """
        self.client_name = client_name
        self.broker_address = broker_address
        self.port = port
        self.username = username
        self.password = password
        self.topic = topic
        self.client = mqtt.Client(client_id=f"{client_name}_{int(time.time())}")
        self.client.username_pw_set(username, password)
        self.client.on_connect = self.on_connect
        self.client.on_disconnect = self.on_disconnect
        self.client_connected = threading.Event()
        self.connected = False

    def connect(self):
        """
        Connects to the MQTT broker.

        Returns:
            bool: True if connection successful, False otherwise.
        """
        if self.connected:
            print(f"{self.client_name} already connected, skipping...")
            return True

        try:
            self.client.connect(self.broker_address, self.port)
            self.client.loop_start()

            if not self.client_connected.wait(timeout=10):
                print(f"{self.client_name} failed to connect within timeout")
                return False

            self.connected = True
            print(f"{self.client_name} connected to broker")
            return True
        except Exception as e:
            print(f"{self.client_name} connection failed: {e}")
            return False

    def disconnect(self):
        """
        Disconnects from the MQTT broker.
        """
        self.client.disconnect()
        self.connected = False
        print(f"{self.client_name} disconnected from broker")

    def publish_heartbeat(self):
        """
        Publishes random heartbeat values to the MQTT broker.
        """
        while True:
            if not self.connected:
                print(f"{self.client_name} is not connected, attempting to reconnect...")
                if not self.connect():
                    time.sleep(5)
                    continue

            heartbeat = random.randint(60, 130)
            try:
                self.client.publish(self.topic, heartbeat)
                print(f"{self.client_name} sent heartbeat: {heartbeat}")
                time.sleep(5)
            except Exception as e:
                print(f"{self.client_name} failed to publish heartbeat: {e}")
                self.connected = False
                time.sleep(5)

    def run(self):
        """
        Runs the MQTT client: connects to the broker and starts publishing heartbeats.
        """
        self.connect()
        self.publish_heartbeat()

    def on_connect(self, client, userdata, flags, rc):
        """
        Callback function called when the MQTT client connects to the broker.

        Args:
            client (mqtt.Client): The MQTT client instance.
            userdata: User data passed during connection.
            flags: Response flags sent by the broker.
            rc (int): Result code of the connection attempt.
        """
        if rc == 0:
            print(f"{self.client_name} connected to broker")
            self.connected = True
            self.client_connected.set()
        else:
            print(f"{self.client_name} connection failed with code {rc}")

    def on_disconnect(self, client, userdata, rc):
        """
        Callback function called when the MQTT client disconnects from the broker.

        Args:
            client (mqtt.Client): The MQTT client instance.
            userdata: User data passed during disconnection.
            rc (int): Result code of the disconnection.
        """
        print(f"{self.client_name} disconnected from broker with code {rc}")
        self.connected = False
        self.client_connected.clear()

# Initialize client information
client_info = [
    {"client_name": "client1", "broker_address": "broker.emqx.io", "port": 1883, "username": "test", "password": "1", "topic": "patient/num1/heartbeat"},
    {"client_name": "client2", "broker_address": "broker.emqx.io", "port": 1883, "username": "test", "password": "1", "topic": "patient/num2/heartbeat"},
    {"client_name": "client3", "broker_address": "broker.emqx.io", "port": 1883, "username": "test", "password": "1", "topic": "patient/num3/heartbeat"},
]

# Create MQTTClient objects and start threads
clients = []

for info in client_info:
    client = MQTTClient(info["client_name"], info["broker_address"], info["port"], info["username"], info["password"], info["topic"])
    clients.append(client)
    client_thread = threading.Thread(target=client.run)
    client_thread.start()

# Keep the main thread running
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Stopping all clients...")
    for client in clients:
        client.disconnect()
    print("All clients stopped.")
